In [1]:
import cv2
import numpy as np 
import pandas as pd 
import os
from PIL import Image
from tensorflow.keras.applications.resnet50 import ResNet50
import tensorflow as tf
import tensorflow.keras as K
from tensorflow.keras import datasets, layers, models, losses, Model
from sklearn.model_selection import train_test_split
from sklearn import preprocessing as pre
from glob import glob
from sklearn.utils.class_weight import compute_class_weight
os.environ["CUDA_VISIBLE_DEVICES"] = '2'


2023-07-27 19:03:22.297682: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-27 19:03:23.082504: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
Train_img_list=pd.read_csv('../../data/standardFrame_data/scale_skip/Train_dataframe.csv')['file_path'].to_list()
Train_label_list=pd.read_csv('../../data/standardFrame_data/scale_skip/Train_dataframe.csv')['standard'].to_list()
Test_img_list=pd.read_csv('../../data/standardFrame_data/scale_skip/Test_dataframe.csv')['file_path'].to_list()
Test_label_list=pd.read_csv('../../data/standardFrame_data/scale_skip/Test_dataframe.csv')['standard'].to_list()
Val_img_list=pd.read_csv('../../data/standardFrame_data/scale_skip/Validation_dataframe.csv')['file_path'].to_list()
Val_label_list=pd.read_csv('../../data/standardFrame_data/scale_skip/Validation_dataframe.csv')['standard'].to_list()

In [3]:
Train_img_path='../../data/standardFrame_data/scale_skip/train'
Test_img_path='../../data/standardFrame_data/scale_skip/test'
Val_img_path='../../data/standardFrame_data/scale_skip/val'

In [4]:
size=256
x_train = np.zeros((len(Train_img_list),size,size,3))
for i in range(len(Train_img_list)):
    x_train[i] =np.array(Image.open(Train_img_path+Train_img_list[i]).resize((size,size)))
x_train=x_train/255
y_train=np.array(Train_label_list)

x_test = np.zeros((len(Test_img_list),size,size,3))
for i in range(len(Test_img_list)):
    x_test[i] =np.array(Image.open(Test_img_path+Test_img_list[i]).resize((size,size)))
x_test=x_test/255
y_test=np.array(Test_label_list)

x_val = np.zeros((len(Val_img_list),size,size,3))
for i in range(len(Val_img_list)):
    x_val[i] =np.array(Image.open(Val_img_path+Val_img_list[i]).resize((size,size)))
x_val=x_val/255
y_val=np.array(Val_label_list)

In [5]:

checkpoint_filepath = "../../model/skip/resnet50_checkpoints.h5"
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_best_only= True
)
class_weight_ratio=compute_class_weight(class_weight = "balanced" , 
                     classes=np.unique(y_train), 
                     y = y_train)
class_weight = {0:class_weight_ratio[0],1:class_weight_ratio[1]}

input_t=K.Input(shape=(size,size, 3))
input_tensor = layers.experimental.preprocessing.Resizing(size, size, interpolation="bilinear", input_shape=x_train.shape[1:])(input_t)
ResNet=ResNet50(include_top=True,weights='imagenet',input_tensor=input_tensor)
model = K.models.Sequential()
model.add(ResNet)
model.add(tf.keras.layers.Dropout(.2, input_shape=(64,)))
model.add(K.layers.Dense(64, activation=tf.keras.layers.LeakyReLU(alpha=0.1)))
model.add(K.layers.Dense(1, activation='sigmoid'))
model.compile(optimizer=K.optimizers.Adam(lr=1e-4),
                loss=tf.keras.losses.binary_crossentropy,
                metrics=["accuracy"])
histo=model.fit(
    x_train,y_train,
    validation_data=(x_val,y_val),
    epochs=500,
    callbacks=[model_checkpoint_callback],
    batch_size=32,shuffle=True,
    class_weight=class_weight
)
model.save('../../model/skip/ResNet50.h5')

2023-07-27 19:05:57.315256: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38163 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:41:00.0, compute capability: 8.0
2023-07-27 19:06:31.324663: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 19557777408 exceeds 10% of free system memory.
2023-07-27 19:06:40.502043: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 19557777408 exceeds 10% of free system memory.


Epoch 1/500


2023-07-27 19:06:59.204340: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8902
2023-07-27 19:07:00.423893: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-07-27 19:07:00.427173: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x7fb6ad584b20 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-07-27 19:07:00.427191: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA A100-PCIE-40GB, Compute Capability 8.0
2023-07-27 19:07:00.430957: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-07-27 19:07:00.551312: I ./tensorflow/compiler/jit/device_compiler.h:180] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the pro

778/778 [==============================] - 110s 101ms/step - loss: 0.5669 - accuracy: 0.6997 - val_loss: 0.2446 - val_accuracy: 0.9335
Epoch 2/500
778/778 [==============================] - 69s 89ms/step - loss: 0.4112 - accuracy: 0.7770 - val_loss: 0.2432 - val_accuracy: 0.9338
Epoch 3/500
778/778 [==============================] - 65s 84ms/step - loss: 0.3502 - accuracy: 0.8078 - val_loss: 0.6744 - val_accuracy: 0.6948
Epoch 4/500
778/778 [==============================] - 65s 83ms/step - loss: 0.3217 - accuracy: 0.8310 - val_loss: 0.3016 - val_accuracy: 0.9335
Epoch 5/500
778/778 [==============================] - 65s 83ms/step - loss: 0.2949 - accuracy: 0.8511 - val_loss: 0.8852 - val_accuracy: 0.6420
Epoch 6/500
778/778 [==============================] - 68s 87ms/step - loss: 0.2616 - accuracy: 0.8702 - val_loss: 0.2188 - val_accuracy: 0.9303
Epoch 7/500
778/778 [==============================] - 64s 83ms/step - loss: 0.2496 - accuracy: 0.8780 - val_loss: 0.3206 - val_accuracy: 0.